In [5]:
import os
import numpy as np

In [2]:
def generate_input(x, y, path):
    data_file = os.path.join(path, 'xy.dat')
    np.savetxt(data_file, np.stack((x, y), axis=-1))
    gnuplot_script = "f(x) = a*x+b\nfit f(x) \"xy.dat\" using 1:2 via a,b"
    gnuplot_file = os.path.join(path, 'input.gnu')
    with open(gnuplot_file, "w") as f:
        f.write(gnuplot_script)

In [3]:
executable = "gnuplot \"input.gnu\""

In [4]:
def _get_gnuplot_parameter_line(content):
    for ii, line in enumerate(content):
        if line.startswith("Final set of parameters"):
            return ii + 2
    raise ValueError("line not found")
    
def get_gnuplot_parameters(content):
    parameter_line = _get_gnuplot_parameter_line(content)
    slope = float(content[parameter_line].split("=")[1].split()[0])
    intercept = float(content[parameter_line + 1].split("=")[1].split()[0])
    return slope, intercept

def parse_output(path):
    data_file = os.path.join(path, 'fit.log')
    with open(data_file, "r") as f:
        content = f.readlines()
    return get_gnuplot_parameters(content)